In [109]:
import pandas as pd
import numpy as np
week1_raw = pd.read_csv("week1_results.csv")
week2_raw = pd.read_csv("Week2_results.csv")
week3_raw = pd.read_csv("week3_results.csv")

In [110]:
#function to combine picks and point values
def combine(df1):
    my_file = df1
    rows = my_file.shape[0]
    i = 2 
    while i < rows:
        for col in my_file.loc[:, my_file.columns != 'Game']:
            my_file[col][i+1] = float(my_file[col][i+1])
            my_file[col][i] = [my_file[col][i], my_file[col][i+1]]
        i = i+2
    return my_file

#function to calculate totals
def totals(df):
    i = 4
    df['Score'] = 0
    df['Game Count'] = 0
    while i < 41:
        for col in df.loc[:, ~df.columns.isin(['Game', 'Score', 'Game Count'])]:
            if df[col][0] == df[col][i][0]:
                #print('hi')
                df.loc[i, 'Game Count'] = df.loc[i, 'Game Count'] + 1
                df.loc[i, 'Score'] = df.loc[i, 'Score'] + df.loc[i, col][1]
        i = i + 2
    df.drop([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41], axis=0, inplace=True)
    return df

#upset counter
def upset_counter(df):
    upset_count = 0
    fav_count = 0
    tie_count = 0
    for col in df.loc[:, ~df.columns.isin(['Game', 'Score'])]:
        if df[col][3] < 0 and df[col][1] == df[col][0]: #if home team is favorite and away team won
            upset_count = upset_count + 1
            print('Road upset alert ', df[col][0], "spread ", -(df[col][3]))
        elif df[col][3] > 0 and df[col][2][0] == df[col][0]: #if away team is favorite and home team won
            upset_count = upset_count + 1
            print('Home dog win ', df[col][2])
        elif df[col][0] == "TIE":
            tie_count = tie_count + 1
            print("A friggen tie...  ", df[col][2])
        else:
            fav_count = fav_count + 1
    print("Total of ", upset_count, "this week.")
    return upset_count, fav_count, tie_count


In [111]:
week1 = combine(week1_raw)
week2 = combine(week2_raw)
week3 = combine(week3_raw)


In [112]:
week1_tot = totals(week1)
week2_tot = totals(week2)
week3_tot = totals(week3)

week1_stats = week1_tot[['Game', 'Score', 'Game Count']]
week2_stats = week2_tot[['Game', 'Score', 'Game Count']]
week3_stats = week3_tot[['Game', 'Score', 'Game Count']]

In [113]:
week1_stats.sort_values(['Score'], ascending=False).head()


,Game,Score,Game Count
12,Kathy Zwartz,97,13
38,Wild Bill,85,10
6,Bob Z,72,11
30,TAZ,71,8
40,Zapp,67,9


In [114]:
week2_stats.sort_values(['Score'], ascending=False).head()


,Game,Score,Game Count
8,Ellieisawesome1,114,14
26,PBW,92,10
6,Bob Z,86,10
40,Zapp,84,10
24,Michael Zwartz,84,9


In [115]:
week3_stats.sort_values(['Score'], ascending=False).head()


,Game,Score,Game Count
18,Lizzy_Z,107,14
16,Lindsey,96,12
4,abfarias,84,8
30,TAZ,75,7
20,Matt McKenna,74,7


In [116]:
#upset count week 1
upset_count1 = upset_counter(week1)[0]



Road upset alert  PIT spread  6.5
Home dog win  ['CHI', 7.0]
Road upset alert  CLE spread  1.5
A friggen tie...   ['HOU', 7.0]
Home dog win  ['MIN', 1.5]
Road upset alert  NYG spread  5.5
Home dog win  ['SEA', 6.5]
Total of  6 this week.


In [117]:
upset_count2 = upset_counter(week2)[0]

Road upset alert  NYJ spread  6.0
Road upset alert  MIA spread  3.5
Home dog win  ['JAX', 4.0]
Road upset alert  ARI spread  5.5
Home dog win  ['DAL', 8.0]
Total of  5 this week.


In [118]:
upset_count3 = upset_counter(week3)[0]


Home dog win  ['TEN', 2.0]
Home dog win  ['MIA', 6.0]
Home dog win  ['IND', 6.5]
Home dog win  ['CAR', 2.5]
Road upset alert  JAX spread  7.0
Road upset alert  GB spread  1.5
Road upset alert  ATL spread  2.0
Home dog win  ['DEN', 1.5]
Road upset alert  DAL spread  1.0
Total of  9 this week.


In [119]:
#Season totals
season_totals_columns = week1_stats
season_totals = season_totals_columns[['Score', 'Game Count']] + week2_stats[['Score', 'Game Count']] + week3_stats[['Score', 'Game Count']]



In [120]:
season_totals.sort_values('Score', ascending=False).head()

,Score,Game Count
12,242,30
8,231,30
18,225,29
16,220,29
6,219,30
